In [ ]:
import pandas as pd
from re import sub

In [ ]:
def preprocess(text):
    text = text.lower().strip()
    text = sub('[áàãâä]', 'a', sub('[éèêë]', 'e', sub('[íìîï]', 'i', sub('[óòõôö]', 'o', sub('[úùûü]', 'u', sub('[ç]','c',text))))))
    text = sub(r'\s+', ' ',text)
    return text

In [ ]:
df_excel = pd.read_excel('troubleshooting_equipament.xlsx')
df_excel.head()

In [ ]:
df = df_excel.astype(str).apply(lambda x: x.apply(preprocess))

### problem

In [ ]:
df_problems = pd.DataFrame(df["problem"].unique(), columns=["problem"])
df_problems['problem_id'] = df_problems.index
display(df_problems.head())
df_problems.shape

### Cause

In [ ]:
df_causes = pd.DataFrame(df["cause"].unique(), columns=["cause"])
df_causes['cause_id'] = df_causes.index
display(df_causes.head())
df_causes.shape

### action

In [ ]:
df_action = pd.DataFrame(df["action"].unique(), columns=["action"])
df_action['action_id'] = df_action.index
display(df_action.head())
df_action.shape

### equipment

In [ ]:
df_equipment = pd.DataFrame(df["equipment"].unique(), columns=["equipment"])
df_equipment['equipment_id'] = df_equipment.index
display(df_equipment.head())
df_equipment.shape

### troubleshooting

In [ ]:
# Criação do DataFrame para a tabela Troubleshooting
df_troubleshooting = df[['type', 'equipment', 'problem', 'cause', 'action']].copy()
df_troubleshooting['equipment_id'] = df_troubleshooting['equipment'].map(df_equipment.set_index('equipment')['equipment_id'])
df_troubleshooting['problem_id'] = df_troubleshooting['problem'].map(df_problems.set_index('problem')['problem_id'])
df_troubleshooting['cause_id'] = df_troubleshooting['cause'].map(df_causes.set_index('cause')['cause_id'])
df_troubleshooting['action_id'] = df_troubleshooting['action'].map(df_action.set_index('action')['action_id'])
df_troubleshooting = df_troubleshooting[['type', 'equipment_id', 'problem_id', 'cause_id', 'action_id']]

df_troubleshooting['equipment_id'] = df_troubleshooting['equipment_id'].astype(int)
df_troubleshooting['problem_id'] = df_troubleshooting['problem_id'].astype(int)
df_troubleshooting['cause_id'] = df_troubleshooting['cause_id'].astype(int)
df_troubleshooting['action_id'] = df_troubleshooting['action_id'].astype(int)

df_troubleshooting[['equipment_id', 'problem_id', 'cause_id', 'action_id']] = df_troubleshooting[['equipment_id', 'problem_id', 'cause_id', 'action_id']].applymap(lambda x: int(x) + 1)

display(df_troubleshooting.head())
df_troubleshooting.shape

### banco

In [ ]:
# Remover a coluna 'id' dos DataFrames
df_problems = df_problems.drop('problem_id', axis=1)
df_causes = df_causes.drop('cause_id', axis=1)
df_action = df_action.drop('action_id', axis=1)
df_equipment = df_equipment.drop('equipment_id', axis=1)

In [ ]:
df_problems

In [ ]:
import pymysql
from sqlalchemy import create_engine

# Configurações de conexão com o banco de dados MySQL
host = 'localhost'
port = 3306
user = 'root'
password = ''
database = 'equipment_failures'

# Cria a conexão com o banco de dados
conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

df_problems.to_sql('problem', engine, if_exists='append', index=False)
df_causes.to_sql('cause', engine, if_exists='append', index=False)
df_action.to_sql('action', engine, if_exists='append', index=False)
df_equipment.to_sql('equipment', engine, if_exists='append', index=False)

df_troubleshooting.to_sql('troubleshooting', engine, if_exists='append', index=False)

conn.close()
